In [0]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [0]:
#importa
#Carros_temp = spark.read.csv("/FileStore/tables/Carros-5.csv",inferSchema=True, header=True, sep=";") 
Carros_temp = spark.table("workspace.carros.carros")
Carros_temp.show(5) # Mostra as 5 primeiras linhas da tabela Carros_temp

In [0]:
#separa colunas
Carros = Carros_temp.select("Consumo","Cilindros","Cilindradas","HP") # Seleciona as colunas Consumo, Cilindros, Cilindradas e HP da tabela Carros_temp
Carros.show(5) # Mostra as 5 primeiras linhas da tabela Carros

In [0]:
#vetoriza atributos
veccaracteristicas = VectorAssembler(inputCols=[("Consumo"),("Cilindros"),("Cilindradas")],outputCol="caracteristicas") # Cria um vetor com as colunas Consumo, Cilindros e Cilindradas e coloca na coluna caracteristicas
Carros = veccaracteristicas.transform(Carros) # Transforma a tabela Carros adicionando a coluna caracteristicas com o vetor das colunas Consumo, Cilindros e Cilindradas
Carros.show(5) # Mostra as 5 primeiras linhas da tabela Carros com a coluna caracteristicas

In [0]:
#divide treino e teste
CarrosTreino, CarrosTeste = Carros.randomSplit([0.7,0.3]) # Divide a tabela Carros em duas tabelas, CarrosTreino e CarrosTeste, com 70% dos dados para treino e 30% para teste
print(CarrosTreino.count()) # Mostra a quantidade de linhas da tabela CarrosTreino
print(CarrosTeste.count()) # Mostra a quantidade de linhas da tabela CarrosTeste

In [0]:
#modelo
reglin = LinearRegression(featuresCol="caracteristicas", labelCol="HP") # Cria o modelo de regressão linear com as colunas caracteristicas e HP
modelo = reglin.fit(CarrosTreino) # Treina o modelo com os dados de treino

In [0]:
#prever
previsao = modelo.transform(CarrosTeste) # Faz a previsão dos valores de HP com os dados de teste
previsao.show(5) # Mostra as 5 primeiras linhas da tabela previsao com os valores previstos de HP

In [0]:
#avaliar performance
avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="HP",metricName='rmse') # Cria o avaliador de performance com as colunas prediction e HP e a métrica RMSE (Root Mean Squared Error)
rmse = avaliar.evaluate(previsao) # Avalia a performance do modelo com os dados de teste e calcula o RMSE
print(rmse) # Mostra o valor do RMSE